In [1]:
!python roou.py --input fitsdata/fits_train --fwhm 1.4 --sig 1.2 --mode 0


In [1]:
#config.py
class Config:
    data_path = "figures"
    model_path_train = ""
    model_path_test = "figures/checkpoint/model_20.ckpt" 
    output_path = "results"

    img_size = 424
    adjust_size = 424
    train_size = 424
    img_channel = 3
    conv_channel_base = 64

    learning_rate = 0.0002
    beta1 = 0.5
    max_epoch = 20
    L1_lambda = 100
    save_per_epoch=1


In [2]:
#utils.py

import numpy as np
import tensorflow as tf
import scipy.misc

def batch_norm(x, scope):
    return tf.contrib.layers.batch_norm(x, decay=0.9, updates_collections=None, epsilon=1e-5, scale=True, scope=scope)

def conv2d(input, output_dim, k_h=4, k_w=4, d_h=2, d_w=2, stddev=0.02, name="conv2d"):
    with tf.variable_scope(name):
        weight = tf.get_variable('weight', [k_h, k_w, input.get_shape()[-1], output_dim],
                            initializer=tf.truncated_normal_initializer(stddev=stddev))
        bias = tf.get_variable('bias', [output_dim], initializer=tf.constant_initializer(0.0))
        conv = tf.nn.bias_add(tf.nn.conv2d(input, weight, strides=[1, d_h, d_w, 1], padding='SAME'), bias)
        return conv

def deconv2d(input, output_shape, k_h=4, k_w=4, d_h=2, d_w=2, stddev=0.02, name="deconv2d"):
    with tf.variable_scope(name):
        weight = tf.get_variable('weight', [k_h, k_w, output_shape[-1], input.get_shape()[-1]],
                            initializer=tf.random_normal_initializer(stddev=stddev))
        bias = tf.get_variable('bias', [output_shape[-1]], initializer=tf.constant_initializer(0.0))
        deconv = tf.nn.bias_add(tf.nn.conv2d_transpose(input, weight, output_shape=output_shape, strides=[1, d_h, d_w, 1]), bias)
        return deconv

def lrelu(x, leak=0.2):
    return tf.maximum(x, leak * x)

def linear(input, output_size, scope=None, stddev=0.02, bias_start=0.0):
    shape = input.get_shape().as_list()
    with tf.variable_scope(scope or "Linear"):
        weight = tf.get_variable("weight", [shape[1], output_size], tf.float32,
                                 tf.random_normal_initializer(stddev=stddev))
        bias = tf.get_variable("bias", [output_size],
                               initializer=tf.constant_initializer(bias_start))
        return tf.matmul(input, weight) + bias

def imread(path):
    return scipy.misc.imread(path)

def imsave(image, path):
    return scipy.misc.imsave(path, image)


In [3]:
#data.py
from config import Config as conf
from utils import imread
import os

def load(path):
    for i in os.listdir(path):
        all = imread(path + "/" + i)
        img, cond = all[:,:conf.img_size], all[:,conf.img_size:]
        yield (img, cond, i)

def load_data():
    data = dict()
    data["train"] = lambda: load(conf.data_path + "/train")
    data["test"] = lambda: load(conf.data_path + "/test")
    return data


In [4]:
#model.py
from config import Config as conf
from utils import conv2d, deconv2d, linear, batch_norm, lrelu
import tensorflow as tf

class CGAN(object):

    def __init__(self):
        self.image = tf.placeholder(tf.float32, shape=(1,conf.img_size, conf.img_size, conf.img_channel))
        self.cond = tf.placeholder(tf.float32, shape=(1,conf.img_size, conf.img_size, conf.img_channel))

        self.gen_img = self.generator(self.cond)

        pos = self.discriminator(self.image, self.cond, False)
        neg = self.discriminator(self.gen_img, self.cond, True)
        pos_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=pos, labels=tf.ones_like(pos)))
        neg_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=neg, labels=tf.zeros_like(neg)))

        self.d_loss = pos_loss + neg_loss
        self.g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=neg, labels=tf.ones_like(neg))) + \
                      conf.L1_lambda * tf.reduce_mean(tf.abs(self.image - self.gen_img))

        t_vars = tf.trainable_variables()
        self.d_vars = [var for var in t_vars if 'disc' in var.name]
        self.g_vars = [var for var in t_vars if 'gen' in var.name]

    def discriminator(self, img, cond, reuse):
        dim = len(img.get_shape())
        with tf.variable_scope("disc", reuse=reuse):
            image = tf.concat([img, cond], dim - 1)
            feature = conf.conv_channel_base
            h0 = lrelu(conv2d(image, feature, name="h0"))
            h1 = lrelu(batch_norm(conv2d(h0, feature*2, name="h1"), "h1"))
            h2 = lrelu(batch_norm(conv2d(h1, feature*4, name="h2"), "h2"))
            h3 = lrelu(batch_norm(conv2d(h2, feature*8, name="h3"), "h3"))
            h4 = linear(tf.reshape(h3, [1,-1]), 1, "linear")
        return h4

    def generator(self, cond):
        with tf.variable_scope("gen"):
            feature = conf.conv_channel_base
            e1 = conv2d(cond, feature, name="e1")
            e2 = batch_norm(conv2d(lrelu(e1), feature*2, name="e2"), "e2")
            e3 = batch_norm(conv2d(lrelu(e2), feature*4, name="e3"), "e3")
            e4 = batch_norm(conv2d(lrelu(e3), feature*8, name="e4"), "e4")
            e5 = batch_norm(conv2d(lrelu(e4), feature*8, name="e5"), "e5")
            e6 = batch_norm(conv2d(lrelu(e5), feature*8, name="e6"), "e6")
            e7 = batch_norm(conv2d(lrelu(e6), feature*8, name="e7"), "e7")
            e8 = batch_norm(conv2d(lrelu(e7), feature*8, name="e8"), "e8")

            size = conf.img_size
            num = [0] * 9
            for i in range(1,9):
                num[9-i]=size
                size =(size+1)/2

            d1 = deconv2d(tf.nn.relu(e8), [1,num[1],num[1],feature*8], name="d1")
            d1 = tf.concat([tf.nn.dropout(batch_norm(d1, "d1"), 0.5), e7], 3)
            d2 = deconv2d(tf.nn.relu(d1), [1,num[2],num[2],feature*8], name="d2")
            d2 = tf.concat([tf.nn.dropout(batch_norm(d2, "d2"), 0.5), e6], 3)
            d3 = deconv2d(tf.nn.relu(d2), [1,num[3],num[3],feature*8], name="d3")
            d3 = tf.concat([tf.nn.dropout(batch_norm(d3, "d3"), 0.5), e5], 3) 
            d4 = deconv2d(tf.nn.relu(d3), [1,num[4],num[4],feature*8], name="d4")
            d4 = tf.concat([batch_norm(d4, "d4"), e4], 3)
            d5 = deconv2d(tf.nn.relu(d4), [1,num[5],num[5],feature*4], name="d5")
            d5 = tf.concat([batch_norm(d5, "d5"), e3], 3) 
            d6 = deconv2d(tf.nn.relu(d5), [1,num[6],num[6],feature*2], name="d6")
            d6 = tf.concat([batch_norm(d6, "d6"), e2], 3)
            d7 = deconv2d(tf.nn.relu(d6), [1,num[7],num[7],feature], name="d7")
            d7 = tf.concat([batch_norm(d7, "d7"), e1], 3) 
            d8 = deconv2d(tf.nn.relu(d7), [1,num[8],num[8],conf.img_channel], name="d8")

            return tf.nn.tanh(d8)


In [5]:
#train.py
from config import Config as conf
from data import *
import scipy.misc
from model import CGAN
from utils import imsave
import tensorflow as tf
import numpy as np
import time
import sys

def prepocess_train(img, cond,):
    img = scipy.misc.imresize(img, [conf.adjust_size, conf.adjust_size])
    cond = scipy.misc.imresize(cond, [conf.adjust_size, conf.adjust_size])
    h1 = int(np.ceil(np.random.uniform(1e-2, conf.adjust_size - conf.train_size)))
    w1 = int(np.ceil(np.random.uniform(1e-2, conf.adjust_size - conf.train_size)))
    img = img[h1:h1 + conf.train_size, w1:w1 + conf.train_size]
    cond = cond[h1:h1 + conf.train_size, w1:w1 + conf.train_size]
    if np.random.random() > 0.5:
        img = np.fliplr(img)
        cond = np.fliplr(cond)
    img = img/127.5 - 1.
    cond = cond/127.5 - 1.
    img = img.reshape(1, conf.img_size, conf.img_size, conf.img_channel)
    cond = cond.reshape(1, conf.img_size, conf.img_size, conf.img_channel)
    return img,cond

def prepocess_test(img, cond):

    img = scipy.misc.imresize(img, [conf.train_size, conf.train_size])
    cond = scipy.misc.imresize(cond, [conf.train_size, conf.train_size])
    img = img.reshape(1, conf.img_size, conf.img_size, conf.img_channel)
    cond = cond.reshape(1, conf.img_size, conf.img_size, conf.img_channel)
    img = img/127.5 - 1.
    cond = cond/127.5 - 1.
    return img,cond

def train():
    data = load_data()
    model = CGAN()

    d_opt = tf.train.AdamOptimizer(learning_rate=conf.learning_rate).minimize(model.d_loss, var_list=model.d_vars)
    g_opt = tf.train.AdamOptimizer(learning_rate=conf.learning_rate).minimize(model.g_loss, var_list=model.g_vars)

    saver = tf.train.Saver()

    start_time = time.time()
    if not os.path.exists(conf.data_path + "/checkpoint"):
        os.makedirs(conf.data_path + "/checkpoint")
    if not os.path.exists(conf.output_path):
        os.makedirs(conf.output_path)

    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    with tf.Session(config=config) as sess:
        if conf.model_path_train == "":
            sess.run(tf.global_variables_initializer())
        else:
            saver.restore(sess, conf.model_path_train)
        for epoch in xrange(conf.max_epoch):
            counter = 0
            train_data = data["train"]()
            for img, cond, name in train_data:
                img, cond = prepocess_train(img, cond)
                _, m = sess.run([d_opt, model.d_loss], feed_dict={model.image:img, model.cond:cond})
                _, m = sess.run([d_opt, model.d_loss], feed_dict={model.image:img, model.cond:cond})
                _, M = sess.run([g_opt, model.g_loss], feed_dict={model.image:img, model.cond:cond})
                counter += 1
                if counter % 50 ==0:
                    print "Epoch [%d], Iteration [%d]: time: %4.4f, d_loss: %.8f, g_loss: %.8f" \
                      % (epoch, counter, time.time() - start_time, m, M)
            if (epoch + 1) % conf.save_per_epoch == 0:
                save_path = saver.save(sess, conf.data_path + "/checkpoint/" + "model_%d.ckpt" % (epoch+1))
                print "Model saved in file: %s" % save_path
                test_data = data["test"]()
                for img, cond, name in test_data:
                    pimg, pcond = prepocess_test(img, cond)
                    gen_img = sess.run(model.gen_img, feed_dict={model.image:pimg, model.cond:pcond})
                    gen_img = gen_img.reshape(gen_img.shape[1:])
                    gen_img = (gen_img + 1.) * 127.5
                    image = np.concatenate((gen_img, cond), axis=1).astype(np.int)
                    imsave(image, conf.output_path + "/%s" % name)

if __name__ == "__main__":
    if len(sys.argv) > 1 and sys.argv[1] == 'gpu=':
        os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
        os.environ["CUDA_VISIBLE_DEVICES"]=str(sys.argv[1][4:])
    else:
        os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
        os.environ["CUDA_VISIBLE_DEVICES"]=str(0)
    train()



AttributeError: 'module' object has no attribute 'imread'

In [ ]:
#test.py
from config import Config as conf
from data import *
import scipy.misc
from model import CGAN
from utils import imsave
import tensorflow as tf
import numpy as np
import time
import sys
from IPython import embed

def prepocess_test(img, cond):

    img = scipy.misc.imresize(img, [conf.train_size, conf.train_size])
    cond = scipy.misc.imresize(cond, [conf.train_size, conf.train_size])
    img = img.reshape(1, conf.img_size, conf.img_size, conf.img_channel)
    cond = cond.reshape(1, conf.img_size, conf.img_size, conf.img_channel)
    img = img/127.5 - 1.
    cond = cond/127.5 - 1.
    return img,cond

def test():

    if not os.path.exists("test"):
        os.makedirs("test")
    data = load_data()
    model = CGAN()

    d_opt = tf.train.AdamOptimizer(learning_rate=conf.learning_rate).minimize(model.d_loss, var_list=model.d_vars)
    g_opt = tf.train.AdamOptimizer(learning_rate=conf.learning_rate).minimize(model.g_loss, var_list=model.g_vars)

    saver = tf.train.Saver()

    counter = 0
    start_time = time.time()

    with tf.Session() as sess:
        saver.restore(sess, conf.model_path_test)
        test_data = data["test"]()
        for img, cond, name in test_data:
            pimg, pcond = prepocess_test(img, cond)
            gen_img = sess.run(model.gen_img, feed_dict={model.image:pimg, model.cond:pcond})
            gen_img = gen_img.reshape(gen_img.shape[1:])
            gen_img = (gen_img + 1.) * 127.5
            image = np.concatenate((gen_img, cond), axis=1).astype(np.int)
            imsave(image, "./test" + "/%s" % name)

if __name__ == "__main__":
    if len(sys.argv) > 1 and sys.argv[1] == 'gpu=':
        os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
        os.environ["CUDA_VISIBLE_DEVICES"]=str(sys.argv[1][4:])
    else:
        os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
        os.environ["CUDA_VISIBLE_DEVICES"]=str(0)
    test()


In [ ]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-
import argparse
import numpy as np
import cv2
import math
import random
from scipy.stats import norm
import matplotlib.pyplot as plt
import os
import pyfits
import glob
from IPython import embed
# mode : 0 training : 1 testing

#parser = argparse.ArgumentParser()

def fspecial_gauss(size, sigma):
    x, y = np.mgrid[-size//2 + 1:size//2 + 1, -size//2 + 1:size//2 + 1]
    g = np.exp(-((x**2 + y**2)/(2.0*sigma**2)))
    return g/g.sum()

def adjust(origin):
    img = origin.copy()
    img[img>4] = 4
    img[img < -0.1] = -0.1
    MIN = np.min(img)
    MAX = np.max(img)
    img = np.arcsinh(10*(img - MIN)/(MAX-MIN))/3
    return img

def roou():
    is_demo = 0
    
    
    parser.add_argument("--fwhm", default="1.4")
    parser.add_argument("--sig", default="1.2")
    parser.add_argument("--input", default="/home/ubuntu/GalaxyGAN_python/fits_train")   #./fits_0.01_0.02
    parser.add_argument("--figure", default="figures")       #./figures/test
    parser.add_argument("--gpu", default = "1")
    parser.add_argument("--model", default = "models")
    parser.add_argument("--mode", default="0")
    
    args = parser.parse_args()
    '''
    args.fwhm = "1.4"
    args.sig = "1.2"
    args.input = "fitsdata/fits_test"
    args.figure = "figures"
    args.mode = "1"
    ''' 
    fwhm = float(args.fwhm)
    sig  = float(args.sig)
    input =  args.input
    figure = args.figure
    mode = int(args.mode)

    train_folder = '%s/train'%(args.figure)
    test_folder = '%s/test'%(args.figure)
    deconv_folder = '%s/deconv'%(args.figure)

    if not os.path.exists('./' + args.figure):
        os.makedirs("./" + args.figure)
    if not os.path.exists("./" + train_folder):
        os.makedirs("./" + train_folder)
    if not os.path.exists("./" + test_folder):
        os.makedirs("./" + test_folder)
    #if not os.path.exists("./" + deconv_folder):
        #os.makedirs("./"+ deconv_folder)

    fits = '%s/*/*-g.fits'%(input)
    files = glob.iglob(fits)

    for i in files:
        #print i
        file_name = os.path.basename(i)

        #readfiles
        if False:
            file_name = '587725489986928743'
            fwhm = 1.4
            sig = 1.2
            mode = 1
            input_folder='fits_0.01_0.02'
            figure_folder='figures'

        filename = file_name.replace("-g.fits", '')
        filename_g = '%s/%s/%s-g.fits'%(input,filename,filename)
        filename_r = '%s/%s/%s-r.fits'%(input,filename,filename)
        filename_i = '%s/%s/%s-i.fits'%(input,filename,filename)

        gfits = pyfits.open(filename_g)
        rfits = pyfits.open(filename_r)
        ifits = pyfits.open(filename_i)
        data_g = gfits[0].data
        data_r = rfits[0].data
        data_i = ifits[0].data

        figure_original = np.ones((data_g.shape[0],data_g.shape[1],3))
        figure_original[:,:,0] = data_g
        figure_original[:,:,1] = data_r
        figure_original[:,:,2] = data_i

        #print figure_original

        if is_demo:
            cv2.imshow("img", adjust(figure_original))
            cv2.waitKey(0)

        # gaussian filter
        fwhm_use = fwhm/0.396
        gaussian_sigma = fwhm_use / 2.355

        # with problem
        figure_blurred = cv2.GaussianBlur(figure_original, (5,5), gaussian_sigma)

        #print "IIIIII"

        if is_demo:
            cv2.imshow("i", figure_blurred)
            cv2.waitKey(0)

        # add white noise
        figure_original_nz =  figure_original[figure_original<0.1]
        figure_original_nearzero = figure_original_nz[figure_original_nz>-0.1]
        figure_blurred_nz = figure_blurred[figure_blurred<0.1]
        figure_blurred_nearzero = figure_blurred_nz[figure_blurred_nz>-0.1]
        [m,s] = norm.fit(figure_original_nearzero)
        [m2,s2] = norm.fit(figure_blurred_nearzero)

        whitenoise_var = (sig*s)**2-s2**2

        if whitenoise_var < 0:
            whitenoise_var = 0.00000001

        whitenoise =  np.random.normal(0, np.sqrt(whitenoise_var) , (data_g.shape[0], data_g.shape[1]))

        figure_blurred[:,:,0] = figure_blurred[:,:,0] + whitenoise
        figure_blurred[:,:,1] = figure_blurred[:,:,1] + whitenoise
        figure_blurred[:,:,2] = figure_blurred[:,:,2] + whitenoise

        if is_demo:
            cv2.imshow('k',figure_blurred)
            cv2.waitKey(0)

        # deconvolution
        if mode>2:
            hsize = 2*np.ceil(2*gaussian_sigma)+1
            PSF = fspecial_gauss(hsize, gaussian_sigma)
            #figure_deconv = deconvblind(figure_blurred, PSF)
            if is_demo:
                cv2.imshow(figure_deconv)
                cv2.waitKey(0)

        # thresold
        MAX = 4
        MIN = -0.1

        figure_original[figure_original<MIN]=MIN
        figure_original[figure_original>MAX]=MAX

        figure_blurred[figure_blurred<MIN]=MIN
        figure_blurred[figure_blurred>MAX]=MAX
        if mode>2:
            figure_deconv[figure_deconv<MIN]=MIN
            figure_deconv[figure_deconv>MAX]=MAX

        # normalize figures
        figure_original = (figure_original-MIN)/(MAX-MIN)
        figure_blurred = (figure_blurred-MIN)/(MAX-MIN)

        if mode:
            figure_deconv = (figure_deconv-MIN)/(MAX-MIN)
            if is_demo:
                plt.subplot(1,3,1), plt.subimage(figure_original), plt.subplot(1,3,2), plt.subimage(figure_blurred),plt.subplot(1,3,3), plt.subimage(figure_deconv)
        elif is_demo:
            plt.subplot(1,2,1), plt.subimage(figure_original), plt.subplot(1,2,2), plt.subimage(figure_blurred)
        

        # asinh scaling
        figure_original = np.arcsinh(10*figure_original)/3
        figure_blurred = np.arcsinh(10*figure_blurred)/3

        if mode>2:
            figure_deconv = np.arcsinh(10*figure_deconv)/3

        # output result to pix2pix format
        figure_combined = np.zeros((data_g.shape[0], data_g.shape[1]*2,3))
        figure_combined[:,: data_g.shape[1],:] = figure_original[:,:,:]
        figure_combined[:, data_g.shape[1]:2*data_g.shape[1],:] = figure_blurred[:,:,:]

        if is_demo:
            cv2.imshow(figure_combined)
            cv2.waitKey(0)

        if mode:
            jpg_path = '%s/test/%s.jpg'% (figure,filename)
        else:
            jpg_path = '%s/train/%s.jpg'% (figure,filename)

        if mode == 2:
            figure_combined_no_ori = np.zeros(data_g.shape[0], data_g.shape[1]*2,3)
            figure_combined_no_ori[:, data_g.shape[1]:2*data_g.shape[1],:] = figure_blurred[:,:,:]
            cv2.imwrite(figure_combined_no_ori,jpg_path)
        else:
            image = (figure_combined*256).astype(np.int)
            cv2.imwrite(jpg_path, image)

        if mode>2:
            deconv_path = '%s/deconv/%s.jpg'% (figure_folder,filename)
            cv2.imwrite(figure_deconv,deconv_path)

#roou()

In [2]:
!python roou.py --input fitsdata/fits_test --fwhm 1.4 --sig 1.2 --mode 1

In [9]:
!python train.py gpu=1

2019-02-09 20:52:03.880969: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Epoch [0], Iteration [50]: time: 87.1439, d_loss: 17.27122879, g_loss: 198.56675720
Epoch [0], Iteration [100]: time: 171.9262, d_loss: 0.00000000, g_loss: 33.84594727
Epoch [0], Iteration [150]: time: 253.9782, d_loss: 0.00000000, g_loss: 115.30859375
Epoch [0], Iteration [200]: time: 336.3782, d_loss: 0.00000000, g_loss: 97.59055328
Epoch [0], Iteration [250]: time: 419.5729, d_loss: 0.00000000, g_loss: 40.26135635
Epoch [0], Iteration [300]: time: 499.1425, d_loss: 0.00000003, g_loss: 13.05464745
Epoch [0], Iteration [350]: time: 580.2997, d_loss: 0.00000033, g_loss: 48.02958679
Epoch [0], Iteration [400]: time: 662.9017, d_loss: 9.69929028, g_loss: 54.71321106
Epoch [0], Iteration [450]: time: 744.5558, d_loss: 0.00000008, g_loss: 56.39681244
Epoch [0], Iteration [500]: time: 825.2161, d_loss: 20.60516930, g_lo

In [8]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [4]:
!python train.py --input fitsdata/fits_train --fwhm 1.4 --sig 1.2 --mode 0

2019-02-08 00:51:13.565232: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
Epoch [0], Iteration [50]: time: 341.4245, d_loss: 0.00000000, g_loss: 136.86447144
Epoch [0], Iteration [100]: time: 679.2703, d_loss: 0.00000000, g_loss: 88.93694305
Epoch [0], Iteration [150]: time: 1016.7976, d_loss: 0.00000000, g_loss: 102.54133606
Epoch [0], Iteration [200]: time: 1354.6129, d_loss: 1.65420246, g_loss: 10.38683701
Epoch [0], Iteration [250]: time: 1692.9260, d_loss: 0.00000000, g_loss: 26.27103233
Epoch [0], Iteration [300]: time: 2030.1929, d_loss: 28.72229767, g_loss: 6.73000336
Epoch [0], Iteration [350]: time: 2368.4239, d_loss: 0.00000000, g_loss: 46.47138596
Epoch [0], Iteration [400]: time: 2705.9719, d_loss: 2.50512075, g_loss: 22.79463768
Epoch [0], Iteration [450]: time: 3043.7451, d_loss: 1.18071461, g_loss: 36.05168915
Epoch [0], I

In [46]:
!python train.py --input fitsdata/fits_train --fwhm 1.4 --sig 1.2 --mode 0

2019-02-08 22:38:01.686732: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
Epoch [0], Iteration [50]: time: 341.5192, d_loss: 0.02631843, g_loss: 6.82035685
Epoch [0], Iteration [100]: time: 856.6463, d_loss: 0.00000000, g_loss: 36.24295807
Epoch [0], Iteration [150]: time: 1417.1216, d_loss: 17.92021561, g_loss: 68.42836761
Epoch [0], Iteration [200]: time: 1978.6674, d_loss: 0.00000882, g_loss: 48.15463638
Epoch [0], Iteration [250]: time: 2534.4049, d_loss: 0.00000000, g_loss: 21.02285385
Epoch [0], Iteration [300]: time: 2873.1356, d_loss: 5.11323738, g_loss: 19.89553833
Epoch [0], Iteration [350]: time: 3213.4300, d_loss: 9.48789501, g_loss: 9.65452194
Epoch [0], Iteration [400]: time: 3553.9240, d_loss: 0.16444132, g_loss: 18.39451790
Epoch [0], Iteration [450]: time: 3894.9185, d_loss: 0.13844803, g_loss: 14.19021797
Epoch [0], Iter

In [ ]:
!python test.py --input fitsdata/fits_test --fwhm 1.4 --sig 1.2 --mode 1

2019-02-09 18:12:06.356413: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.


In [4]:
import numpy as np

In [35]:
np.ceil(np.random.uniform(1e-2, 0))

1.0

In [39]:
a =np.ones([4,4])

In [47]:
print(a[1:5:1:5])

SyntaxError: invalid syntax (<ipython-input-47-5f24b276c675>, line 1)